### build datasets from FASHION_MNIST

In [12]:
%matplotlib inline
import sys
import d2lzh as d2l
from mxnet.gluon import data as gdata
from mxnet import autograd, nd

batch_size = 256
transformer = gdata.vision.transforms.ToTensor()
mnist_train, mnist_test = gdata.vision.FashionMNIST(train=True), gdata.vision.FashionMNIST(train=False)

if sys.platform.startswith('win'):
    num_workers = 1
else:
    num_workers = 4
    
####
!echo num_worker: {num_workers} 
####

train_iter, test_iter = \
    gdata.DataLoader(mnist_train.transform_first(transformer), batch_size, shuffle=True, num_workers=num_workers), \
    gdata.DataLoader(mnist_test.transform_first(transformer), batch_size, shuffle=True, num_workers=num_workers)

num_inputs, num_outputs = 28 * 28, 10 # each image's size = 28^2 pixel
W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
b = nd.zeros((1, num_outputs))

W.attach_grad()
b.attach_grad()

print(W, b)

num_worker: 4

[[ 4.5648916e-03 -4.1309558e-03 -6.7057782e-03 ...  2.1987283e-03
  -1.1190008e-03  3.7409394e-04]
 [-4.5822975e-03  5.8028642e-03  2.5114496e-03 ... -7.1809615e-04
  -3.0394192e-03  5.2552321e-04]
 [-9.5167458e-03 -2.6465463e-03  2.4275223e-04 ... -1.3457411e-03
  -9.6758017e-03 -2.1776833e-02]
 ...
 [-6.2633078e-03  1.4432447e-02 -4.0996582e-03 ...  2.1082694e-03
   1.8460415e-03  3.9075227e-03]
 [-1.5023038e-02 -6.8704103e-04 -2.4936814e-02 ...  4.7025760e-03
  -5.8409167e-03 -7.9640791e-05]
 [ 8.8863180e-04  1.3216679e-02  5.0818673e-03 ...  6.1003589e-03
   5.5059949e-03  1.4979360e-03]]
<NDArray 784x10 @cpu(0)> 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
<NDArray 1x10 @cpu(0)>


In [14]:
X = nd.array([[1, 2, 3],[4, 5, 6]])
print(X.sum(axis=0, keepdims=True), X.sum(axis=1, keepdims=True))



[[5. 7. 9.]]
<NDArray 1x3 @cpu(0)> 
[ 6. 15.]
<NDArray 2 @cpu(0)>


1
